<a href="https://colab.research.google.com/github/Saroj12Dangol/Document-Scanner/blob/Grammar-Correction/transformer_based_grammar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install happytransformer

In [ ]:
from happytransformer import HappyTextToText
import nltk
nltk.download('punkt')
from datasets import load_dataset
import csv
from happytransformer import TTTrainArgs
from happytransformer import TTSettings



happy_tt = HappyTextToText("T5", "t5-base")

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
replacements = [
  (" .", "."), 
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!")
]

def remove_excess_spaces(text):from datasets import load_dataset

  for rep in replacements:
    text = text.replace(rep[0], rep[1])

  return text

In [ ]:
train_dataset = load_dataset("jfleg", split='validation[:]')
train_dataset = train_dataset.filter(lambda example: example['sentence'] != ' ')


eval_dataset = load_dataset("jfleg", split='test[:]')
eval_dataset = eval_dataset.filter(lambda example: example['sentence'] != ' ')

Filter:   0%|          | 0/755 [00:00<?, ? examples/s]

Filter:   0%|          | 0/748 [00:00<?, ? examples/s]

In [ ]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [ ]:
for case in train_dataset["corrections"]:
  print(case)
  print(case[0])
  print("--------------------------------------------------------")

In [ ]:
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input 
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases contain blank strings. 
                if input_text and correction:
                  # input_text = remove_excess_spaces(input_text)
                  # correction = remove_excess_spaces(correction)
                  writter.writerow([input_text, correction])
    

In [ ]:

generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)

In [ ]:
before_result = happy_tt.eval("eval.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7f4abcdcb5f3d2ee/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


In [ ]:
print("Before loss:", before_result.loss)

Before loss: 1.0288490056991577


In [ ]:


args = TTTrainArgs(batch_size=8, num_train_epochs=12, max_input_length=2, max_output_length=2048)
train_results = happy_tt.train("train.csv", args=args)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5f50ba4b60c6e1b8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/3016 [00:00<?, ? examples/s]

PyTorch: setting up devices
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3016
  Num Epochs = 12
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4524
  Number of trainable parameters = 222903552


Step,Training Loss
500,1.138800
1000,1.071600
1500,1.023500
2000,0.979000
2500,0.964300
3000,0.959800
3500,0.955100
4000,0.966800
4500,0.992000




Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:


before_loss = happy_tt.eval("eval.csv",args=args)



print("After loss: ", before_loss.loss)

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 8


After loss:  4.361475944519043


In [ ]:
happy_tt.save("model/grammar")

In [3]:
input_text_1 ="""Imagine a vast sheep of paper on which straight
Lines, Inceangles, Squares, Pentagons, Haveragons,
and other figures, instead of remaining fixed in
their places, more freely, about, on on in the surface
had without the power of aiming above on sinking
below it, very much like shadows - only hand and
with luminious edges - and you will then have a pretty
correct motion of my country and countrymen. Asas, a
few years ago, I should have said " my universe : but
over my minds has been opened to higher views of
things.
"""


In [ ]:
from happytransformer import HappyTextToText
grammar = HappyTextToText("T5", "/content/drive/MyDrive/model/grammar")

In [14]:
result = ''
for sentence in sentences:
  output_text_3 = grammar.generate_text("grammar: "+sentence, args=beam_settings)
  result += output_text_3.text +" "
print(result)

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Imagine a vast sheep of paper on which straight Lines, Inceangles, Squares, Pentagons, Haveragons, and other figures, instead of remaining fixed in their places, more freely, about, on on in the surface had without the power of aiming above on sinking below it, very much like shadows - only hand and with luminious edges - and you will then have a pretty correct motion of my country and countrymen . Asas , a few years ago I should have said " my universe : but over my minds has been opened to higher views of things . " 


In [ ]:
grammar = HappyTextToText("T5", "/content/drive/MyDrive/model/grammar")
beam_settings =  TTSettings(num_beams=3, min_length=2, max_length=2048)

In [18]:
def grammar_correction(grammar, paragraph):
  beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=600)
  sentences = nltk.sent_tokenize(paragraph)
  result = ''
  for sentence in sentences:
    output_text_3 = grammar.generate_text("grammar: "+sentence, args=beam_settings)
    result += output_text_3.text +" "
  print(result)
  

In [19]:
text = f"""Imagine a vast sheep of paper on which straight
Lines, Inceangles, Squares, Pentagons, Haveragons,
and other figures, instead of remaining fixed in
their places, more freely, about, on on in the surface,
had without the power of aiming above on sinking
below it, very much like shadows - only hand and
with luminious edges - and you will then have a pretty
correct motion of my country and countrymen. Asas, a
few years ago, I should have said " my universe : but
over my minds has been opened to higher views of
things.
"""
grammar_correction(grammar,text)

Imagine a vast sheep of paper on which straight Lines, Inceangles, Squares, Pentagons, Haveragons, and other figures, instead of remaining fixed in their places, more freely, about, on on in the surface, had without the power of aiming above on sinking below it, very much like shadows - only hand and with luminious edges - and you will then have a pretty correct motion of my country and countrymen . Asas , a few years ago I should have said " my universe : but over my minds has been opened to higher views of things . " 
